# Es.1 - Conceptual Similarity

In questo esercizio andiamo ad esplorare varie tecniche per calcolare la similarità semantica tra 
due parole. Per farlo utilizzeremo il dataset WordSim353.csv, che contiene una serie di coppie di
 parole e il loro punteggio di similarità. Il punteggio è un valore reale compreso tra 0 e 10.

Le misure di similarità che utilizzeremo sono:

- Wu and Palmer
- Shortest Path
- Leakcock & Chodorow

### Imports

In [10]:
from nltk.corpus import stopwords
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
import random

### Data preparation

In [11]:
def get_text_from_file(path):
    '''
    Read a file and, after revoving all stopwords, return a list of words.
    '''
    file = []
    stop_words = set(stopwords.words('english'))
    with open (path, 'r') as f:
        for row in f:
            filtered_s = [w for w in word_tokenize(row) if not w.lower() in stop_words]
            file.append(simple_preprocess(str(filtered_s), deacc=True))
    f.close()
    return file

def extract_triple(path):
    '''
    Read a file and return the first 2 word in each row, as a tuple.
    '''
    tuple_list = []
    with open (path, 'r') as f:
        for row in f:
            # Remove \n
            row = row.strip()
            # Organize in a triple the values
            tuple_list.append(tuple(row.split(",")[:3]))
            
        # Remove first value of the tuple
        tuple_list.pop(0)
    f.close()
    return tuple_list

In [12]:
values = extract_triple('../data/WordSim353.csv')

### Wu and Palmer

In [17]:
def get_synsets(term):
    '''
    Retrurn the synsets of a term.
    '''
    if(len(wn.synsets(term)) > 0):
        return wn.synsets(term)
    return None

def depth(syn):
    '''
    Return the distance between the WorNet root and a synset.
    '''
    if(syn is None):
        return 0
    if(type(syn) is list):
        if(len(syn) > 0):
            return syn[0].max_depth()
        return 0
    return syn.max_depth()

def lcs(syn1, syn2):
    '''
    Return the longest common subsumer of two synsets.
    '''
    if(syn1 is None or syn2 is None):
        return None
    return syn1.lowest_common_hypernyms(syn2)

def wu_pal_sim(syn1, syn2):
    '''
    Return the Wu-Palmer similarity of two synsets.
    '''
    if(syn1 is None or syn2 is None):
        return 0
    return syn1.wup_similarity(syn2)

def my_wu_pal_sim(syn1, syn2):
    if(syn1 is None or syn2 is None):
        return None
    else:
        if(depth(syn1) + depth(syn2) == 0):
            return 0
        return 2.0 * depth(lcs(syn1, syn2)) / (depth(syn1) + depth(syn2))
    
def max_similarity(syns1, syns2):
    sim_max = ("", "", 0)
    
    if(syns1 is not None or syns2 is not None):
        return sim_max
        for syn1 in syns1:
            for syn2 in syns2:
                if(syns1 is not None and syns2 is not None):
                    sim = my_wu_pal_sim(syn1, syn2)
                    if sim > sim_max[2]:
                        sim_max = (syn1, syn2, sim)
                    
    return sim_max

In [18]:
for val in values:
    syns1 = get_synsets(val[0])
    syns2 = get_synsets(val[1])
    
    print(syns1, syns2)
        
    sim = max_similarity(syns1, syns2)

    print(sim, "given value:", val[2])


[Synset('love.n.01'), Synset('love.n.02'), Synset('beloved.n.01'), Synset('love.n.04'), Synset('love.n.05'), Synset('sexual_love.n.02'), Synset('love.v.01'), Synset('love.v.02'), Synset('love.v.03'), Synset('sleep_together.v.01')] [Synset('sexual_activity.n.01'), Synset('sex.n.02'), Synset('sex.n.03'), Synset('sex.n.04'), Synset('arouse.v.07'), Synset('sex.v.02')]
('', '', 0) given value: 6.77
[Synset('tiger.n.01'), Synset('tiger.n.02')] [Synset('cat.n.01'), Synset('guy.n.01'), Synset('cat.n.03'), Synset('kat.n.01'), Synset('cat-o'-nine-tails.n.01'), Synset('caterpillar.n.02'), Synset('big_cat.n.01'), Synset('computerized_tomography.n.01'), Synset('cat.v.01'), Synset('vomit.v.01')]
('', '', 0) given value: 7.35
[Synset('tiger.n.01'), Synset('tiger.n.02')] [Synset('tiger.n.01'), Synset('tiger.n.02')]
('', '', 0) given value: 10.00
[Synset('book.n.01'), Synset('book.n.02'), Synset('record.n.05'), Synset('script.n.01'), Synset('ledger.n.01'), Synset('book.n.06'), Synset('book.n.07'), Syns

### Shortest Path

In [ ]:
def max_path(): #20
    max_path = 0
    for synset in wn.all_synsets():
        if synset.max_depth() > max_path:
            max_path = synset.max_depth()
    return max_path

def max_path_2(): #19
    return max(max(len(hyp_path) for hyp_path in ss.hypernym_paths()) for ss in wn.all_synsets())

max_depth = max_path()